In [1]:
import sys
from pathlib import Path
from os import getcwd
sys.path.append(str(Path(getcwd()).parent))

from datetime import date, timedelta
import codecs
import pandas as pd

pd.set_option('display.max_columns', None)

from os.path import join

from bulletin import default_input, default_output
from bulletin.services.metabase import Metabase
from bulletin.casos_confirmados import CasosConfirmados
from bulletin.utils.static import Municipios
from bulletin.utils.normalize import normalize_hash, normalize_labels, normalize_text, date_hash, normalize_number
from bulletin.utils import static
from bulletin.utils.xls_ import fit_cols

from datetime import datetime


---------------------------------------------------------------

root: C:\SESA\covid19datascience\bulletin
parent: C:\SESA\covid19datascience
default_input: C:\SESA\covid19datascience\input
default_output: C:\SESA\covid19datascience\output

---------------------------------------------------------------



In [2]:
municipios = Municipios()
municipios['mun_resid'] = municipios['municipio'].apply(normalize_text)
municipios.loc[municipios['uf']!='PR','mun_resid'] = municipios.loc[municipios['uf']!='PR','municipio'].apply(normalize_text) + '/' + municipios['uf']

In [3]:
mb = Metabase()
diario = mb.get_downloaded_query('old_diario')
diario

Cookie:'3508d********-****-****-****-************2602c'

saved_queries: ['all_notifica', 'diario', 'old_diario'] 

Load downloaded query time elepsed:00:00:00


'C:\\SESA\\covid19datascience\\input\\queries\\old_diario.csv'

In [4]:
date_parser = lambda x: pd.to_datetime(x, errors='coerce', format='%d/%m/%Y')

notifica = pd.read_csv(diario,
                       sep=',',
                       converters = {
                           'paciente': normalize_text,
                           'origem': lambda x: normalize_number(x,fill=0),
                           'nome_mae': normalize_text,
                           'idade': lambda x: normalize_number(x,fill=0),
                           'pais_residencia': lambda x: normalize_number(x,fill=0),
                           'uf_residencia': lambda x: normalize_number(x,fill=0),
                           'ibge_residencia': lambda x: normalize_number(x,fill=999999),
                           'uf_unidade_notifica': lambda x: normalize_number(x,fill=0),
                           'ibge_unidade_notifica': lambda x: normalize_number(x,fill=999999),
                           'criterio_classificacao': lambda x: normalize_number(x,fill=0),
                           'evolucao': lambda x: normalize_number(x,fill=3) }
                       )


notifica['data_cura_obito'] = notifica['data_cura_obito'].apply(date_parser)
notifica['data_liberacao'] = notifica['data_liberacao'].apply(date_parser)

notifica.loc[notifica['evolucao']==5, 'evolucao'] = 2
notifica.loc[~notifica['evolucao'].isin([1,2,3]),'evolucao'] = 3

notifica['hash'] = (notifica['paciente'].apply(normalize_hash) +
                  notifica['idade'].astype(str) +
                  notifica['ibge_residencia'].astype(str) )


notifica['hash_atend'] = (notifica['paciente'].apply(normalize_hash) +
                  notifica['idade'].astype(str) +
                  notifica['ibge_unidade_notifica'].astype(str) )

notifica['hash_less'] = ( notifica['paciente'].apply(normalize_hash) +
                        notifica['idade'].apply(lambda x: str(x - 1)) +
                        notifica['ibge_residencia'].astype(str) )

notifica['hash_more'] = ( notifica['paciente'].apply(normalize_hash) +
                        notifica['idade'].apply(lambda x: str(x + 1)) +
                        notifica['ibge_residencia'].astype(str) )

notifica['hash_diag'] = notifica['paciente'].apply(normalize_hash) + notifica['data_liberacao'].apply(date_hash)

notifica = pd.merge(notifica.rename(columns={'ibge_residencia':'ibge'}),municipios[['ibge','rs','mun_resid','uf','municipio']],on='ibge',how='left').rename(columns={'ibge':'ibge_residencia'})
notifica = pd.merge(notifica.rename(columns={'ibge_unidade_notifica':'ibge'}),municipios[['ibge','mun_resid']].rename(columns={'mun_resid':'mun_atend'}),on='ibge',how='left').rename(columns={'ibge':'ibge_unidade_notifica'})

notifica = notifica.loc[~(
    (notifica['data_liberacao'].isna()) |
    (notifica['sexo'].isna())
)]

notifica = notifica.loc[(
    (notifica['data_liberacao'] < pd.to_datetime(datetime.today())) &
    (notifica['data_liberacao'] > pd.to_datetime(datetime.strptime('12/03/2020','%d/%m/%Y')))
)]

notifica.groupby('evolucao')[['id']].count()

,id
evolucao,
1,19739
2,1176
3,26537


In [5]:
cc = CasosConfirmados()
cc.load('cc_28_05_2021')
casosct = cc.df
casosct.tail()

databases: ['cc_27_05_2021', 'cc_28_05_2021', 'cc_29_05_2021']
Load C:\SESA\covid19datascience\bulletin\database\casos_confirmados\cc_28_05_2021.pkl
loading Casos Confirmados from pkl time elepsed:00:00:51


,identificacao,id_notifica,uf_resid,ibge_resid,ibge_atend,nome,sexo,idade,laboratorio,dt_diag,comunicacao,is,evolucao,data_evolucao,data_com_evolucao,hash,hash_atend,hash_less,hash_more,hash_diag
1084375,1176616,-1,PR,411820,999999,ZENILTON PIRES MODESTO,M,39,"COVID-19, Biologia Molecular",2021-05-17,2021-05-28,NaN,3.0,NaT,NaT,ZENILTONPIRESMODESTO39411820,ZENILTONPIRESMODESTO39999999,ZENILTONPIRESMODESTO38411820,ZENILTONPIRESMODESTO40411820,ZENILTONPIRESMODESTO17052021
1084376,1176617,-1,PR,410720,410720,ZEUNILDE AGUIAR ACERMAO,F,37,"COVID-19, Biologia Molecular",2021-05-27,2021-05-28,2021-05-17 00:00:00,3.0,NaT,NaT,ZEUNILDEAGUIARACERMAO37410720,ZEUNILDEAGUIARACERMAO37410720,ZEUNILDEAGUIARACERMAO36410720,ZEUNILDEAGUIARACERMAO38410720,ZEUNILDEAGUIARACERMAO27052021
1084377,1176618,-1,PR,411840,411840,ZILDA LOPES,F,48,"COVID-19, Biologia Molecular",2021-05-16,2021-05-28,2021-05-09 00:00:00,3.0,NaT,NaT,ZILDALOPES48411840,ZILDALOPES48411840,ZILDALOPES47411840,ZILDALOPES49411840,ZILDALOPES16052021
1084378,1176619,-1,PR,411840,411840,ZILDA MACHADO PAZZETTO DOS SANTOS,F,42,Teste Rápido Antígeno (Imunocromatografia),2021-05-27,2021-05-28,2021-05-24 00:00:00,3.0,NaT,NaT,ZILDAMACHADOPAZZETTODOSSANTOS42411840,ZILDAMACHADOPAZZETTODOSSANTOS42411840,ZILDAMACHADOPAZZETTODOSSANTOS41411840,ZILDAMACHADOPAZZETTODOSSANTOS43411840,ZILDAMACHADOPAZZETTODOSSANTOS27052021
1084379,1176620,-1,PR,411920,410970,ZULEIDE MENDES DE ALMEIDA,F,55,"COVID-19, Biologia Molecular",2021-05-26,2021-05-28,2021-05-21 00:00:00,3.0,NaT,NaT,ZULEIDEMENDESDEALMEIDA55411920,ZULEIDEMENDESDEALMEIDA55410970,ZULEIDEMENDESDEALMEIDA54411920,ZULEIDEMENDESDEALMEIDA56411920,ZULEIDEMENDESDEALMEIDA26052021


In [6]:
novos_casos = notifica.loc[~(
    (notifica['hash'].isin(casosct['hash'])) |
    (notifica['hash'].isin(casosct['hash_less'])) |
    (notifica['hash'].isin(casosct['hash_more'])) |
    (notifica['hash_diag'].isin(casosct['hash_diag']))
)].copy()

novos_casos['data_com'] = pd.to_datetime(date.today()).strftime('%d/%m/%Y')
novos_casos = novos_casos.sort_values('paciente')
novos_casos

,id,origem,data_notificacao,updated_at,paciente,sexo,nome_mae,idade,data_nascimento,pais_residencia,uf_residencia,ibge_residencia,uf_unidade_notifica,ibge_unidade_notifica,exame,criterio_classificacao,data_liberacao,data_coleta,data_recebimento,data_1o_sintomas,evolucao,data_cura_obito,hash,hash_atend,hash_less,hash_more,hash_diag,rs,mun_resid,uf,municipio,mun_atend,data_com
70158,3946446,1,30/03/2021,29/05/2021,ABEGAIL CASTANHO COELHO,F,ANTONIA PEDROSO DA SILVA,86,28/07/1934,1,41,411820,41,411820,Vírus Respiratório,5,2021-04-01,30/03/2021,31/03/2021,24/03/2021,1,2021-04-04,ABEGAILCASTANHOCOELHO86411820,ABEGAILCASTANHOCOELHO86411820,ABEGAILCASTANHOCOELHO85411820,ABEGAILCASTANHOCOELHO87411820,ABEGAILCASTANHOCOELHO01042021,1,PARANAGUA,PR,Paranaguá,PARANAGUA,29/05/2021
28368,3753806,1,19/05/2021,28/05/2021,ABEL DE SOUZA,M,ROSI APARECIDA DE SOUZA,42,27/11/1978,1,41,410580,41,999999,"COVID-19, Biologia Molecular",1,2021-05-22,16/05/2021,22/05/2021,18/05/2021,3,NaT,ABELDESOUZA42410580,ABELDESOUZA42999999,ABELDESOUZA41410580,ABELDESOUZA43410580,ABELDESOUZA22052021,2,COLOMBO,PR,Colombo,IGNORADO/EXTERIOR/IG,29/05/2021
49899,3854525,1,25/05/2021,28/05/2021,ABEL LANKOVSKI,M,TECLA PORANIAK,71,07/01/1950,1,41,410170,41,999999,"COVID-19, Biologia Molecular",1,2021-05-28,25/05/2021,27/05/2021,22/05/2021,3,NaT,ABELLANKOVSKI71410170,ABELLANKOVSKI71999999,ABELLANKOVSKI70410170,ABELLANKOVSKI72410170,ABELLANKOVSKI28052021,11,ARARUNA,PR,Araruna,IGNORADO/EXTERIOR/IG,29/05/2021
56732,3880086,1,26/05/2021,29/05/2021,ABIMAEL FERNANDES GUIMARAES,M,ASENATH ALVES GUIMARAES,43,19/10/1977,1,41,411125,41,411125,"COVID-19, Biologia Molecular",1,2021-05-29,26/05/2021,28/05/2021,23/05/2021,3,NaT,ABIMAELFERNANDESGUIMARAES43411125,ABIMAELFERNANDESGUIMARAES43411125,ABIMAELFERNANDESGUIMARAES42411125,ABIMAELFERNANDESGUIMARAES44411125,ABIMAELFERNANDESGUIMARAES29052021,2,ITAPERUCU,PR,Itaperuçu,ITAPERUCU,29/05/2021
25834,3733934,1,18/05/2021,28/05/2021,ABNER GILMAR DA SILVA SANTOS,M,NILCEIA APARECIDA DA SILVA,20,02/08/2000,1,41,412240,41,412240,"COVID-19, Biologia Molecular",1,2021-05-27,21/05/2021,27/05/2021,17/05/2021,3,NaT,ABNERGILMARDASILVASANTOS20412240,ABNERGILMARDASILVASANTOS20412240,ABNERGILMARDASILVASANTOS19412240,ABNERGILMARDASILVASANTOS21412240,ABNERGILMARDASILVASANTOS27052021,17,ROLANDIA,PR,Rolândia,ROLANDIA,29/05/2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20069,3680672,1,15/05/2021,28/05/2021,ZILDA APARECIDA BARBOZA BATISTA,F,ADELIA DE OLIVEIRA,58,17/08/1962,1,41,411840,41,411840,"COVID-19, Biologia Molecular",1,2021-05-18,15/05/2021,18/05/2021,11/05/2021,1,2021-05-28,ZILDAAPARECIDABARBOZABATISTA58411840,ZILDAAPARECIDABARBOZABATISTA58411840,ZILDAAPARECIDABARBOZABATISTA57411840,ZILDAAPARECIDABARBOZABATISTA59411840,ZILDAAPARECIDABARBOZABATISTA18052021,14,PARANAVAI,PR,Paranavaí,PARANAVAI,29/05/2021
31716,3777427,2,20/05/2021,28/05/2021,ZILDA MEDEIROS DOS SANTOS,F,ALBERTINA ALVES DUQUE,61,20/01/1960,1,41,410690,41,410690,"COVID-19, Biologia Molecular",1,2021-05-22,20/05/2021,21/05/2021,13/05/2021,3,NaT,ZILDAMEDEIROSDOSSANTOS61410690,ZILDAMEDEIROSDOSSANTOS61410690,ZILDAMEDEIROSDOSSANTOS60410690,ZILDAMEDEIROSDOSSANTOS62410690,ZILDAMEDEIROSDOSSANTOS22052021,2,CURITIBA,PR,Curitiba,CURITIBA,29/05/2021
65527,3923416,1,28/05/2021,29/05/2021,ZILTON ANTONIO LEITE,M,SEBASTIANA LEITE,31,04/07/1989,1,41,410850,41,410850,Teste Rápido Antígeno (Imunocromatografia),1,2021-05-29,29/05/2021,29/05/2021,23/05/2021,3,NaT,ZILTONANTONIOLEITE31410850,ZILTONANTONIOLEITE31410850,ZILTONANTONIOLEITE30410850,ZILTONANTONIOLEITE32410850,ZILTONANTONIOLEITE29052021,6,GENERAL CARNEIRO,PR,General Carneiro,GENERAL CARNEIRO,29/05/2021
51767,3861326,1,25/05/2021,28/05/2021,ZORAIDE ESTEFANO DA SILVA,F,LEONILDA ELEUTERIA DOS SANTOS,71,10/12/1949,1,41,411915,41,999999,"COVID-19, Biologia Molecular",1,2021-05-27,25/05/2021,26/05/2021,23/05/2021,3,NaT,ZORAIDEESTEFANODASILVA71411915,ZORAIDEESTEFANODASILVA

In [7]:
novos_casos_errados = novos_casos.loc[(
    (novos_casos['data_liberacao'] <= pd.to_datetime('2020-03-01')) |
    (novos_casos['data_liberacao'] > pd.to_datetime(date.today()))
)]

novos_casos = novos_casos.loc[~(
    (novos_casos['data_liberacao'] <= pd.to_datetime('2020-03-01')) |
    (novos_casos['data_liberacao'] > pd.to_datetime(date.today()))
)]

novos_casos[['id','paciente','sexo','idade','mun_resid', 'mun_atend', 'rs', 'exame','data_liberacao','data_com','data_1o_sintomas','hash']].to_excel(join(default_output,'novos_casos.xlsx'))

novos_casos_errados

,id,origem,data_notificacao,updated_at,paciente,sexo,nome_mae,idade,data_nascimento,pais_residencia,uf_residencia,ibge_residencia,uf_unidade_notifica,ibge_unidade_notifica,exame,criterio_classificacao,data_liberacao,data_coleta,data_recebimento,data_1o_sintomas,evolucao,data_cura_obito,hash,hash_atend,hash_less,hash_more,hash_diag,rs,mun_resid,uf,municipio,mun_atend,data_com


In [8]:
obitos_notifica = notifica.loc[notifica['evolucao']==2]
obitos_casos = casosct.loc[casosct['evolucao']==2]

novos_obitos = obitos_notifica.loc[~(
    (obitos_notifica['hash'].isin(obitos_casos['hash'])) |
    (obitos_notifica['hash'].isin(obitos_casos['hash_less'])) |
    (obitos_notifica['hash'].isin(obitos_casos['hash_more'])) |
    (obitos_notifica['hash_diag'].isin(obitos_casos['hash_diag'])) 
)]

novos_obitos = novos_obitos.loc[novos_obitos['data_cura_obito'].notna()]
novos_obitos = novos_obitos.loc[novos_obitos['hash'].isin(casosct['hash'].tolist() + novos_casos['hash'].tolist())]
novos_obitos = novos_obitos.sort_values('paciente')
novos_obitos

,id,origem,data_notificacao,updated_at,paciente,sexo,nome_mae,idade,data_nascimento,pais_residencia,uf_residencia,ibge_residencia,uf_unidade_notifica,ibge_unidade_notifica,exame,criterio_classificacao,data_liberacao,data_coleta,data_recebimento,data_1o_sintomas,evolucao,data_cura_obito,hash,hash_atend,hash_less,hash_more,hash_diag,rs,mun_resid,uf,municipio,mun_atend
10820,3545839,2,06/05/2021,28/05/2021,ADEMAR LIMA DA SILVA,M,LIELZA LIMA DA SILVA,42,13/12/1978,1,41,410690,41,410690,"COVID-19, Biologia Molecular",1,2021-05-08,06/05/2021,07/05/2021,04/05/2021,2,2021-05-26,ADEMARLIMADASILVA42410690,ADEMARLIMADASILVA42410690,ADEMARLIMADASILVA41410690,ADEMARLIMADASILVA43410690,ADEMARLIMADASILVA08052021,2,CURITIBA,PR,Curitiba,CURITIBA
31605,3776927,2,20/05/2021,28/05/2021,ADEMIR LOUZADA,M,ROSELI FRANCISCO LOUZADA,38,11/03/1983,1,41,410690,41,410690,Vírus Respiratório,1,2021-05-25,21/05/2021,24/05/2021,09/05/2021,2,2021-05-24,ADEMIRLOUZADA38410690,ADEMIRLOUZADA38410690,ADEMIRLOUZADA37410690,ADEMIRLOUZADA39410690,ADEMIRLOUZADA25052021,2,CURITIBA,PR,Curitiba,CURITIBA
10768,3544384,10,06/05/2021,29/05/2021,ADIMARIO ANTONIO DA SILVA,M,ANTONIA SIQUEIRA,66,18/12/1954,1,41,411520,41,411520,Teste Rápido Antígeno (Imunocromatografia),1,2021-05-03,03/05/2021,03/05/2021,01/05/2021,2,2021-05-28,ADIMARIOANTONIODASILVA66411520,ADIMARIOANTONIODASILVA66411520,ADIMARIOANTONIODASILVA65411520,ADIMARIOANTONIODASILVA67411520,ADIMARIOANTONIODASILVA03052021,15,MARINGA,PR,Maringá,MARINGA
13047,3597386,2,10/05/2021,28/05/2021,AIDA MARITZA CHAVEZ TORREALBA,F,CACILDA RAMONA TORREALBA DE CHAVEZ,54,26/09/1966,1,41,410690,41,410690,"COVID-19, Biologia Molecular",1,2021-05-13,11/05/2021,12/05/2021,06/05/2021,2,2021-05-24,AIDAMARITZACHAVEZTORREALBA54410690,AIDAMARITZACHAVEZTORREALBA54410690,AIDAMARITZACHAVEZTORREALBA53410690,AIDAMARITZACHAVEZTORREALBA55410690,AIDAMARITZACHAVEZTORREALBA13052021,2,CURITIBA,PR,Curitiba,CURITIBA
46962,3844351,1,24/05/2021,28/05/2021,ALEX ALEXANDRE DE PAIVA,M,MARIA ELZA PEREIRA,42,29/10/1978,1,41,411410,41,411410,Teste Rápido Antígeno (Imunocromatografia),1,2021-05-24,24/05/2021,24/05/2021,19/05/2021,2,2021-05-27,ALEXALEXANDREDEPAIVA42411410,ALEXALEXANDREDEPAIVA42411410,ALEXALEXANDREDEPAIVA41411410,ALEXALEXANDREDEPAIVA43411410,ALEXALEXANDREDEPAIVA24052021,15,MANDAGUACU,PR,Mandaguaçu,MANDAGUACU
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14120,3613110,1,11/05/2021,28/05/2021,WASTHI JANSEN,F,CLARA LIDIA JANSEN,80,22/10/1940,1,41,411990,41,411990,"COVID-19, Biologia Molecular",1,2021-02-16,12/02/2021,13/02/2021,09/05/2021,2,2021-05-28,WASTHIJANSEN80411990,WASTHIJANSEN80411990,WASTHIJANSEN79411990,WASTHIJANSEN81411990,WASTHIJANSEN16022021,3,PONTA GROSSA,PR,Ponta Grossa,PONTA GROSSA
7355,3332255,1,19/04/2021,29/05/2021,WESLEY FERNANDO LESSA DE GOES,M,ANTONIA LESSA DE SOUSA GOES,41,21/04/1980,1,41,410420,41,410420,Teste Rápido Antígeno (Imunocromatografia),1,2021-04-19,19/04/2021,19/04/2021,17/04/2021,2,2021-05-28,WESLEYFERNANDOLESSADEGOES41410420,WESLEYFERNANDOLESSADEGOES41410420,WESLEYFERNANDOLESSADEGOES40410420,WESLEYFERNANDOLESSADEGOES42410420,WESLEYFERNANDOLESSADEGOES19042021,2,CAMPO LARGO,PR,Campo Largo,CAMPO LARGO
6745,3263086,1,13/04/2021,28/05/2021,WILSON ADEMARIO VIDAL,M,ANALIA DA PIEDADE VIDAL,66,18/04/1955,1,41,410580,41,999999,"COVID-19, Biologia Molecular",1,2021-04-15,13/04/2021,14/04/2021,05/04/2021,2,2021-05-11,WILSONADEMARIOVIDAL66410580,WILSONADEMARIOVIDAL66999999,WILSONADEMARIOVIDAL65410580,WILSONADEMARIOVIDAL67410580,WILSONADEMARIOVIDAL15042021,2,COLOMBO,PR,Colombo,IGNORADO/EXTERIOR/IG
17667,3655933,2,13/05/2021,28/05/2021,WILSON RODRIGUES IANISCH,M,CASTURINA ADINIR RODRIGUES IANISCH,58,03/04/1963,1,41,410690,41,410690,"COVID-19, Biologia Molecular",1,2021-05-15,13/05/2021,14/05/2021,11/05/2021,2,2021-05-25,WILSONRODRIGUESIANISCH58410690,WILSONRODRIGUESIANISCH58410690,WILSONRODRIGUESIANISCH57410690,WILSONRODRIGUESIANISCH59410690,WILSON

In [9]:
novos_obitos_errados = novos_obitos.loc[(
    (novos_obitos['data_cura_obito'] <= pd.to_datetime('2020-03-01')) |
    (novos_obitos['data_cura_obito'] > pd.to_datetime(date.today()))
)]

novos_obitos = novos_obitos.loc[~(
    (novos_obitos['data_cura_obito'] <= pd.to_datetime('2020-03-01')) |
    (novos_obitos['data_cura_obito'] > pd.to_datetime(date.today()))
)]

novos_obitos[['id','paciente','sexo','idade','mun_resid', 'rs', 'data_cura_obito','hash']].to_excel(join(default_output,'novos_obitos.xlsx'))

novos_obitos_errados

,id,origem,data_notificacao,updated_at,paciente,sexo,nome_mae,idade,data_nascimento,pais_residencia,uf_residencia,ibge_residencia,uf_unidade_notifica,ibge_unidade_notifica,exame,criterio_classificacao,data_liberacao,data_coleta,data_recebimento,data_1o_sintomas,evolucao,data_cura_obito,hash,hash_atend,hash_less,hash_more,hash_diag,rs,mun_resid,uf,municipio,mun_atend
10542,3535459,1,05/05/2021,28/05/2021,PEDRO RIBEIRO MARTINS,M,EMILIA RIBEIRO MARTINS,70,12/08/1950,1,41,411010,41,411990,Vírus Respiratório,1,2021-05-10,06/05/2021,07/05/2021,19/04/2021,2,2021-08-28,PEDRORIBEIROMARTINS70411010,PEDRORIBEIROMARTINS70411990,PEDRORIBEIROMARTINS69411010,PEDRORIBEIROMARTINS71411010,PEDRORIBEIROMARTINS10052021,4,IMBITUVA,PR,Imbituva,PONTA GROSSA


In [10]:
casos_confirmados = pd.merge(casosct.rename(columns={'ibge_resid':'ibge'}),municipios[['ibge','rs']],how='left',on='ibge')

casos_confirmadosPR = casos_confirmados.loc[casos_confirmados['rs']!=99]

obitos_confirmados =  casos_confirmados.loc[casos_confirmados['evolucao']==2]

obitos_confirmadosPR = obitos_confirmados.loc[obitos_confirmados['rs']!=99]

print(f"Total de casos: {len(casos_confirmados)} + {len(novos_casos)}")
print(f"Total de obitos: {len(obitos_confirmados)} + {len(novos_obitos)}\n\n")

novos_casosPR = novos_casos.loc[novos_casos['rs']!=99].copy()
print(f"Total de casos PR: {len(casos_confirmadosPR)} + {len(novos_casosPR)}")

novos_obitosPR = novos_obitos.loc[novos_obitos['rs']!=99].copy()
print(f"Total de obitos PR: {len(obitos_confirmadosPR)} + {len(novos_obitosPR)}")

novos_casosFora = novos_casos.loc[novos_casos['rs']==99].copy()
print(f"Total de casos Fora: {len(casos_confirmados) - len(casos_confirmadosPR)} + {len(novos_casosFora)}")

novos_obitosFora = novos_obitos.loc[novos_obitos['rs']==99].copy()
print(f"Total de obitos Fora: {len(obitos_confirmados) - len(obitos_confirmadosPR)} + {len(novos_obitosFora)}")


novos_obitosPR_group = novos_obitosPR.groupby(by='municipio')

today = datetime.today()
ontem = today - timedelta(1)
anteontem = ontem - timedelta(1)
data_retroativos = ontem - timedelta(14)

retroativos = novos_casosPR.loc[(novos_casosPR['data_liberacao'].apply(date_parser) <= data_retroativos)].sort_values(by='data_liberacao')
last2weeks = novos_casosPR.loc[(novos_casosPR['data_liberacao'].apply(date_parser) > data_retroativos) & (novos_casosPR['data_liberacao'].apply(date_parser) <= anteontem)].sort_values(by='data_liberacao')
hoje = novos_casosPR.loc[(novos_casosPR['data_liberacao'].apply(date_parser) > anteontem)].sort_values(by='data_liberacao')

obitos_retroativos = novos_obitosPR.loc[(novos_obitosPR['data_cura_obito'].apply(date_parser) <= data_retroativos)].sort_values(by='data_cura_obito')
obitos_last2weeks = novos_obitosPR.loc[(novos_obitosPR['data_cura_obito'].apply(date_parser) > data_retroativos) & (novos_obitosPR['data_cura_obito'].apply(date_parser) <= anteontem)].sort_values(by='data_cura_obito')
obitos_hoje = novos_obitosPR.loc[(novos_obitosPR['data_cura_obito'].apply(date_parser) > anteontem)].sort_values(by='data_cura_obito')

relatorio_file = join(default_output,f"relatorio_{(today.strftime('%d/%m/%Y_%Hh').replace('/','_').replace(' ',''))}.txt")
with codecs.open(relatorio_file,"w","utf-8-sig") as relatorio:
    relatorio.write(f"{today.strftime('%d/%m/%Y')}\n")
    relatorio.write(f"{len(novos_casosPR):,} novos casos residentes ".replace(',','.'))

    if len(novos_casosFora) > 0:
        relatorio.write(f"e {len(novos_casosFora):,} não residente{'s' if len(novos_casosFora) > 1 else ''} ".replace(',','.'))
    relatorio.write(f"divulgados no PR.\n")

    relatorio.write(f"{len(casos_confirmadosPR)+len(novos_casosPR):,} casos confirmados residentes do PR.\n".replace(',','.'))
    relatorio.write(f"{len(casos_confirmados)+len(novos_casos):,} total geral.\n\n".replace(',','.'))
    relatorio.write(f"{len(novos_obitosPR):,} Óbitos residentes do PR:\n".replace(',','.'))

    for municipio, obitos in novos_obitosPR_group:
        relatorio.write(f"{len(obitos):,} {municipio}\n".replace(',','.'))

    if len(novos_obitosFora) > 0:
        relatorio.write('\n')
        relatorio.write(f"{len(novos_obitosFora):,} Óbito{'s' if len(novos_obitosFora) > 1 else ''} não residente{'s' if len(novos_obitosFora) > 1 else ''} do PR.\n".replace(',','.'))

    relatorio.write('\n')
    relatorio.write(f"{len(obitos_confirmadosPR)+len(novos_obitosPR):,} óbitos residentes do PR.\n".replace(',','.'))
    relatorio.write(f"{len(obitos_confirmados)+len(novos_obitos):,} total geral.\n\n".replace(',','.'))

    for _, row in novos_obitos.iterrows():
       relatorio.write(f"{row['sexo']}\t{row['idade']}\t{row['municipio'] if row['rs']!=99 else (row['municipio']+'/'+row['uf'])}\t{str(row['rs']).zfill(2) if row['rs']!=99 else '#N/D'}\t{row['data_cura_obito'].day}/{static.meses[row['data_cura_obito'].month-1]}/{row['data_cura_obito'].year}\n")
    relatorio.write('\n')

    if True:

        #casos
        relatorio.write(f"{len(novos_casosPR):,} novos casos residentes divulgados no PR.\n".replace(',','.'))

        relatorio.write(f"{len(retroativos):,} casos retroativos confirmados no período de {retroativos.iloc[0]['data_liberacao'].strftime('%d/%m/%Y')} à {retroativos.iloc[-1]['data_liberacao'].strftime('%d/%m/%Y')}.\n".replace(',','.'))
        relatorio.write(f"{len(last2weeks):,} novos casos confirmados no período de {last2weeks.iloc[0]['data_liberacao'].strftime('%d/%m/%Y')} à {last2weeks.iloc[-1]['data_liberacao'].strftime('%d/%m/%Y')}.\n".replace(',','.'))
        relatorio.write(f"{len(hoje):,} novos casos confirmados hoje.\n\n".replace(',','.'))


        novos_casosPR['month'] = novos_casosPR.apply(lambda x: x['data_liberacao'].month, axis=1)
        novos_casosPR['year'] = novos_casosPR.apply(lambda x: x['data_liberacao'].year, axis=1)
        relatorio.write('Novos casos por meses:\n')
        for group, value in novos_casosPR.groupby(by=['year','month']):
            relatorio.write(f"{static.meses[int(group[1])-1]}\\{group[0]}: {len(value)}\n")
        relatorio.write('\n')

        #obitos
        relatorio.write(f"{len(novos_obitosPR):,} novos obitos residentes divulgados no PR.\n".replace(',','.'))

        relatorio.write(f"{len(obitos_retroativos):,} obitos retroativos ocorridos no período de {obitos_retroativos.iloc[0]['data_cura_obito'].strftime('%d/%m/%Y')} à {obitos_retroativos.iloc[-1]['data_cura_obito'].strftime('%d/%m/%Y')}.\n".replace(',','.'))
        relatorio.write(f"{len(obitos_last2weeks):,} novos obitos ocorridos no período de {obitos_last2weeks.iloc[0]['data_cura_obito'].strftime('%d/%m/%Y')} à {obitos_last2weeks.iloc[-1]['data_cura_obito'].strftime('%d/%m/%Y')}.\n".replace(',','.'))
        relatorio.write(f"{len(obitos_hoje):,} novos obitos ocorridos hoje.\n\n".replace(',','.'))

        relatorio.write('Novos obitos por meses:\n')
        novos_obitosPR['month'] = novos_obitosPR.apply(lambda x: x['data_cura_obito'].month, axis=1)
        novos_obitosPR['year'] = novos_obitosPR.apply(lambda x: x['data_cura_obito'].year, axis=1)
        for group, value in novos_obitosPR.groupby(by=['year','month']):
            relatorio.write(f"{static.meses[int(group[1])-1]}\\{group[0]}: {len(value)}\n")
        relatorio.write('\n')

        relatorio.write('Novos obitos por dia:\n')
        for group, value in novos_obitosPR.groupby(by='data_cura_obito'):
            relatorio.write(f"{group.strftime('%d/%m/%Y')}: {len(value)}\n")

        #-----RELATÓRIO DA COMUNICAÇÃO--------------
        obitos_list = []
        munic = []
        for municipio, obitos in novos_obitosPR_group:
            obito = len(obitos)
            obitos_list.append(obito)
            munic.append(municipio)             

        dicionario = (dict(zip(list(munic),list(obitos_list))))
        #print(dicionario)
        dicionario = sorted(dicionario.items(),key=lambda x: x[1], reverse = True)
        #print(dicionario)


        relatorio.write(f"\nOs pacientes que foram a óbito residiam em: ")
        for municip, obit in dict(dicionario).items():
            if obit != 1:
                relatorio.write(f"{municip} ({obit})")
                relatorio.write(f", ")
        relatorio.write(f".\n")
        relatorio.write(f"A Sesa registra ainda a morte de uma pessoa que residia em cada um dos seguintes municípios:")
        for municip, obit in dict(dicionario).items():
            if obit == 1:
                relatorio.write(f"{municip}")
                relatorio.write(f", ")


Total de casos: 1084380 + 5397
Total de obitos: 26168 + 202


Total de casos PR: 1078504 + 5385
Total de obitos PR: 26021 + 201
Total de casos Fora: 5876 + 12
Total de obitos Fora: 147 + 1


In [11]:
with codecs.open(relatorio_file,"r","utf-8-sig") as relatorio:
    print(relatorio.read())

29/05/2021
5.385 novos casos residentes e 12 não residentes divulgados no PR.
1.083.889 casos confirmados residentes do PR.
1.089.777 total geral.

201 Óbitos residentes do PR:
1 Almirante Tamandaré
2 Araucária
1 Barracão
1 Boa Vista da Aparecida
1 Brasilândia do Sul
1 Campo Largo
1 Campo Magro
1 Campo Mourão
1 Campo do Tenente
1 Candói
1 Capitão Leônidas Marques
6 Cascavel
2 Castro
3 Cidade Gaúcha
6 Colombo
2 Conselheiro Mairinck
91 Curitiba
1 Céu Azul
1 Dois Vizinhos
1 Fazenda Rio Grande
1 Floraí
2 Foz do Iguaçu
5 Francisco Beltrão
2 Guaraniaçu
3 Guarapuava
2 Guaíra
1 Imbaú
1 Imbituva
1 Ipiranga
2 Irati
2 Kaloré
3 Londrina
1 Mandaguaçu
1 Mandirituba
1 Mangueirinha
1 Marialva
2 Maringá
2 Matinhos
1 Medianeira
2 Moreira Sales
1 Ortigueira
1 Ouro Verde do Oeste
1 Palmital
1 Paranaguá
1 Paraíso do Norte
2 Pinhais
1 Pinhão
2 Piraquara
2 Piraí do Sul
2 Ponta Grossa
1 Pontal do Paraná
1 Pérola
1 Quatro Barras
3 Quedas do Iguaçu
1 Reserva
2 Rondon
1 Santa Helena
1 Saudade do Iguaçu
9 São Jos

A Sesa registra ainda a morte de uma pessoa que residia em cada um dos seguintes municípios:Almirante Tamandaré, Barracão, Boa Vista da Aparecida, Brasilândia do Sul, Campo Largo, Campo Magro, Campo Mourão, Campo do Tenente, Candói, Capitão Leônidas Marques, Céu Azul, Dois Vizinhos, Fazenda Rio Grande, Floraí, Imbaú, Imbituva, Ipiranga, Mandaguaçu, Mandirituba, Mangueirinha, Marialva, Medianeira, Ortigueira, Ouro Verde do Oeste, Palmital, Paranaguá, Paraíso do Norte, Pinhão, Pontal do Paraná, Pérola, Quatro Barras, Reserva, Santa Helena, Saudade do Iguaçu, São Miguel do Iguaçu, Tijucas do Sul, Tunas do Paraná, Vera Cruz do Oeste, 


In [12]:
# while input('bateu?') != 'bateu': pass

In [13]:
novos_casos = novos_casos.rename(columns={
    'id':'id_notifica',
    'ibge_residencia': 'ibge_resid',
    'ibge_unidade_notifica': 'ibge_atend',
    'paciente':'nome',
    'sexo': 'sexo',
    'idade':'idade',
    'exame': 'laboratorio',
    'data_liberacao':'dt_diag',
    'data_com':'comunicacao',
    'data_1o_sintomas': 'is',
    'evolucao': 'evolucao',
    'data_cura_obito': 'data_evolucao',
    'uf': 'uf_resid'
})

novos_casos = novos_casos[['id_notifica', 'uf_resid', 'ibge_resid', 'ibge_atend', 'nome', 'sexo', 'idade', 'laboratorio', 'dt_diag', 'comunicacao', 'is', 'evolucao', 'data_evolucao']]

novos_casos.loc[novos_casos['evolucao']==5,'evolucao'] = 2
novos_casos.loc[~novos_casos['evolucao'].isin([1,2,3]),'evolucao'] = 3

novos_casos['hash'] = (novos_casos['nome'].apply(normalize_hash) +
                  novos_casos['idade'].astype(str) +
                  novos_casos['ibge_resid'].astype(str) )

novos_casos['hash_less'] = ( novos_casos['nome'].apply(normalize_hash) +
                        novos_casos['idade'].apply(lambda x: str(x - 1)) +
                        novos_casos['ibge_resid'].astype(str) )

novos_casos['hash_more'] = ( novos_casos['nome'].apply(normalize_hash) +
                        novos_casos['idade'].apply(lambda x: str(x + 1)) +
                        novos_casos['ibge_resid'].astype(str) )

novos_casos['identificacao'] = list(range(casosct.iloc[-1,0] + 1,casosct.iloc[-1,0] + 1 + len(novos_casos)))
casosct = casosct.append(novos_casos)
print(len(casosct))
casosct.groupby('evolucao')[['sexo']].count()

1089777


,sexo
evolucao,
1.0,756194
2.0,26197
3.0,307386


In [14]:
novos_obitos = novos_obitos.rename(columns={
    'id': 'id_notifica',
    'data_cura_obito': 'data_evolucao',
    'paciente':'nome'
})

novos_obitos['hash'] = (novos_obitos['nome'].apply(normalize_hash) +
                          novos_obitos['idade'].astype(str) +
                              novos_obitos['ibge_residencia'].astype(str) )

novos_obitos['data_com_evolucao'] = pd.to_datetime(date.today())
novos_obitos['evolucao'] = 2

novos_obitos = novos_obitos[['hash','id_notifica','evolucao','data_evolucao','data_com_evolucao']]
novos_obitos = novos_obitos.set_index('hash')
casosct = casosct.set_index('hash')
casosct.update(novos_obitos)
casosct.reset_index(inplace=True)
casosct = casosct[['identificacao','id_notifica','uf_resid','ibge_resid','ibge_atend','nome','sexo','idade','laboratorio','dt_diag','comunicacao','is','evolucao','data_evolucao','data_com_evolucao','hash','hash_less','hash_more','hash_diag']]

print(len(casosct))
casosct.groupby('evolucao')[['sexo']].count()

1089777


,sexo
evolucao,
1.0,756178
2.0,26370
3.0,307229


In [15]:
# casosct['is'] = casosct['is'].apply(lambda x: str(x)[:10] if not str(x).lower() in ['none','nat','nan'] else 'none')
# casosct.loc[casosct['is'].str.contains('/'),'is2'] = casosct.loc[casosct['is'].str.contains('/'),'is'].apply(lambda x: pd.to_datetime(x,format='%d/%m/%Y',errors='raise'))
# casosct.loc[casosct['is'].str.contains('-'),'is2'] = casosct.loc[casosct['is'].str.contains('-'),'is'].apply(lambda x: pd.to_datetime(x,format='%Y-%m-%d',errors='raise'))

# casosct['comunicacao'] = casosct['comunicacao'].apply(str)
# casosct.loc[casosct['comunicacao'].str.contains('/'),'com'] = casosct.loc[casosct['comunicacao'].str.contains('/'),'comunicacao'].apply(lambda x: pd.to_datetime(x,format='%d/%m/%Y',errors='raise'))
# casosct.loc[casosct['comunicacao'].str.contains('-'),'com'] = casosct.loc[casosct['comunicacao'].str.contains('-'),'comunicacao'].apply(lambda x: pd.to_datetime(x,format='%Y-%m-%d',errors='raise'))

# casosct['data_evolucao'] = casosct['data_evolucao'].apply(str)
# casosct.loc[casosct['data_evolucao'].str.contains('/'),'evo'] = casosct.loc[casosct['data_evolucao'].str.contains('/'),'data_evolucao'].apply(lambda x: pd.to_datetime(x,format='%d/%m/%Y',errors='coerce'))
# casosct.loc[casosct['data_evolucao'].str.contains('-'),'evo'] = casosct.loc[casosct['data_evolucao'].str.contains('-'),'data_evolucao'].apply(lambda x: pd.to_datetime(x,format='%Y-%m-%d',errors='coerce'))

# casosct['id_notifica'] = casosct['id_notifica'].apply(int)
# casosct['evolucao'] = casosct['evolucao'].apply(int)
# casosct['is'] = casosct['is2']
# casosct['comunicacao'] = casosct['com']
# casosct['data_evolucao'] = casosct['evo'].apply(lambda x: pd.to_datetime(x))

# casosct.dtypes

In [16]:
cc.df = casosct
cc.save('cc_29_05_2021', replace=True)
casosct.tail()

Load C:\SESA\covid19datascience\bulletin\database\casos_confirmados\cc_29_05_2021.pkl
saving Casos Confirmados to pkl time elepsed:00:00:05


,identificacao,id_notifica,uf_resid,ibge_resid,ibge_atend,nome,sexo,idade,laboratorio,dt_diag,comunicacao,is,evolucao,data_evolucao,data_com_evolucao,hash,hash_less,hash_more,hash_diag
1089772,1182013,3680672.0,PR,411840,411840,ZILDA APARECIDA BARBOZA BATISTA,F,58,"COVID-19, Biologia Molecular",2021-05-18,29/05/2021,11/05/2021,1.0,2021-05-28 00:00:00,NaT,ZILDAAPARECIDABARBOZABATISTA58411840,ZILDAAPARECIDABARBOZABATISTA57411840,ZILDAAPARECIDABARBOZABATISTA59411840,NaN
1089773,1182014,3777427.0,PR,410690,410690,ZILDA MEDEIROS DOS SANTOS,F,61,"COVID-19, Biologia Molecular",2021-05-22,29/05/2021,13/05/2021,3.0,NaT,NaT,ZILDAMEDEIROSDOSSANTOS61410690,ZILDAMEDEIROSDOSSANTOS60410690,ZILDAMEDEIROSDOSSANTOS62410690,NaN
1089774,1182015,3923416.0,PR,410850,410850,ZILTON ANTONIO LEITE,M,31,Teste Rápido Antígeno (Imunocromatografia),2021-05-29,29/05/2021,23/05/2021,3.0,NaT,NaT,ZILTONANTONIOLEITE31410850,ZILTONANTONIOLEITE30410850,ZILTONANTONIOLEITE32410850,NaN
1089775,1182016,3861326.0,PR,411915,999999,ZORAIDE ESTEFANO DA SILVA,F,71,"COVID-19, Biologia Molecular",2021-05-27,29/05/2021,23/05/2021,3.0,NaT,NaT,ZORAIDEESTEFANODASILVA71411915,ZORAIDEESTEFANODASILVA70411915,ZORAIDEESTEFANODASILVA72411915,NaN
1089776,1182017,3900570.0,PR,411520,411520,ZULEIDE PEREIRA DE CARVALHO,F,47,"COVID-19, Biologia Molecular",2021-05-28,29/05/2021,20/05/2021,3.0,NaT,NaT,ZULEIDEPEREIRADECARVALHO47411520,ZULEIDEPEREIRADECARVALHO46411520,ZULEIDEPEREIRADECARVALHO48411520,NaN


In [ ]:
#recuperados

In [22]:
municipios = Municipios()

macros_idx_labels = dict([ [i,x] for (i,x) in enumerate(['Leste','Oeste','Noroeste','Norte','Fora'])])
macros_labels_idx = dict([ [x,i] for (i,x) in macros_idx_labels.items() ])
municipios['macro'] = municipios['macro'].apply(lambda x: macros_labels_idx[x])

casosct = pd.merge(casosct.rename(columns={'ibge_resid':'ibge'}),municipios,on=['ibge'],how='left').rename(columns={'ibge':'ibge_resid'})

casosct['municipio_pr'] = casosct['municipio']
casosct.loc[casosct['uf_resid']!='PR','municipio_pr'] = 'Fora'

casosct.loc[casosct['uf_resid']!='PR','ibge7'] = 9999999

casosct['uf_pr'] = 'PR'
casosct.loc[casosct['uf_resid']!='PR','uf_pr'] = 'Fora'

,identificacao,id_notifica,uf_resid,ibge_resid,ibge_atend,nome,sexo,idade,laboratorio,dt_diag,comunicacao,is,evolucao,data_evolucao,data_com_evolucao,hash,hash_less,hash_more,hash_diag,ibge7,rs,regional,macro,municipio,uf,populacao,estado,municipio_pr,uf_pr
0,8838,-1,PR,410550,410550,ELIANA CRISTINA CUNHA DE OLIVEIRA SOUZA,F,47,LACEN,2020-03-12,2020-03-12 00:00:00,NaN,1,2020-04-03 00:00:00,NaT,ELIANACRISTINACUNHADEOLIVEIRASOUZA47410550,ELIANACRISTINACUNHADEOLIVEIRASOUZA46410550,ELIANACRISTINACUNHADEOLIVEIRASOUZA48410550,ELIANACRISTINACUNHADEOLIVEIRASOUZA12032020,4105508,13,Cianorte,2,Cianorte,PR,83816,Paraná,Cianorte,PR
1,10932,-1,PR,410690,410690,FLAMARION PETRIV DE ABREU,M,43,LACEN,2020-03-12,2020-03-12 00:00:00,2020-03-07 00:00:00,1,NaN,NaT,FLAMARIONPETRIVDEABREU43410690,FLAMARIONPETRIVDEABREU42410690,FLAMARIONPETRIVDEABREU44410690,FLAMARIONPETRIVDEABREU12032020,4106902,2,Metropolitana,0,Curitiba,PR,1948626,Paraná,Curitiba,PR
2,10933,-1,PR,410690,410690,FLAMARION PETRIV DE ABREU JUNIOR,M,15,LACEN,2020-03-12,2020-03-12 00:00:00,2020-03-08 00:00:00,1,NaN,NaT,FLAMARIONPETRIVDEABREUJUNIOR15410690,FLAMARIONPETRIVDEABREUJUNIOR14410690,FLAMARIONPETRIVDEABREUJUNIOR16410690,FLAMARIONPETRIVDEABREUJUNIOR12032020,4106902,2,Metropolitana,0,Curitiba,PR,1948626,Paraná,Curitiba,PR
3,11470,-1,PR,410690,410690,GABRIELA FUGANTI CASAGRANDE,F,26,Genoprimer,2020-03-16,2020-03-12 00:00:00,2020-03-11 00:00:00,1,NaN,NaT,GABRIELAFUGANTICASAGRANDE26410690,GABRIELAFUGANTICASAGRANDE25410690,GABRIELAFUGANTICASAGRANDE27410690,GABRIELAFUGANTICASAGRANDE16032020,4106902,2,Metropolitana,0,Curitiba,PR,1948626,Paraná,Curitiba,PR
4,11492,-1,PR,410690,410690,GABRIELA SCHNEIDER GUGELMIN,F,26,Genoprimer,2020-03-18,2020-03-12 00:00:00,2020-03-15 00:00:00,1,NaN,NaT,GABRIELASCHNEIDERGUGELMIN26410690,GABRIELASCHNEIDERGUGELMIN25410690,GABRIELASCHNEIDERGUGELMIN27410690,GABRIELASCHNEIDERGUGELMIN18032020,4106902,2,Metropolitana,0,Curitiba,PR,1948626,Paraná,Curitiba,PR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1089772,1182013,3.680.672,PR,411840,411840,ZILDA APARECIDA BARBOZA BATISTA,F,58,"COVID-19, Biologia Molecular",2021-05-18,29/05/2021,11/05/2021,1,2021-05-28 00:00:00,NaT,ZILDAAPARECIDABARBOZABATISTA58411840,ZILDAAPARECIDABARBOZABATISTA57411840,ZILDAAPARECIDABARBOZABATISTA59411840,NaN,4118402,14,Paranavaí,2,Paranavaí,PR,88922,Paraná,Paranavaí,PR
1089773,1182014,3.777.427,PR,410690,410690,ZILDA MEDEIROS DOS SANTOS,F,61,"COVID-19, Biologia Molecular",2021-05-22,29/05/2021,13/05/2021,3,NaT,NaT,ZILDAMEDEIROSDOSSANTOS61410690,ZILDAMEDEIROSDOSSANTOS60410690,ZILDAMEDEIROSDOSSANTOS62410690,NaN,4106902,2,Metropolitana,0,Curitiba,PR,1948626,Paraná,Curitiba,PR
1089774,1182015,3.923.416,PR,410850,410850,ZILTON ANTONIO LEITE,M,31,Teste Rápido Antígeno (Imunocromatografia),2021-05-29,29/05/2021,23/05/2021,3,NaT,NaT,ZILTONANTONIOLEITE31410850,ZILTONANTONIOLEITE30410850,ZILTONANTONIOLEITE32410850,NaN,4108502,6,União da Vitória,0,General Carneiro,PR,13685,Paraná,General Carneiro,PR
1089775,1182016,3.861.326,PR,411915,999999,ZORAIDE ESTEFANO DA SILVA,F,71,"COVID-19, Biologia Molecular",2021-05-27,29/05/2021,23/05/2021,3,NaT,NaT,ZORAIDEESTEFANODASILVA71411915,ZORAIDEESTEFANODASILVA70411915,ZORAIDEESTEFANODASILVA72411915,NaN,4119152,2,Metropolitana,0,Pinhais,PR,133490,Paraná,Pinhais,PR


In [23]:
pd.options.display.float_format = lambda x: '{:,.0f}'.format(x).replace(',','.')

total = casosct.groupby(['evolucao','uf_pr'])[['sexo']].count().unstack().droplevel(0,1)[['PR','Fora']]
total['GERAL'] = total.sum(1)
total = total.append(pd.Series(total.sum(0),name=0))
total.index = [2,1,3,0]
total = total.sort_index()
total = total.rename(index={0:'CASOS',2:'RECUPERADOS',1:'OBITOS',3:'ATIVOS'})
total.index.name = None
total.columns.name = None

total.astype(float)

,PR,Fora,GERAL
CASOS,1.083.889,5.888,1.089.777
OBITOS,26.222,148,26.370
RECUPERADOS,753.963,2.215,756.178
ATIVOS,303.704,3.525,307.229


In [24]:
total_macros = casosct.groupby(['macro','evolucao'])[['sexo']].count().unstack()
total_macros.columns = total_macros.columns.droplevel(0)

total_macros['CASOS'] = total_macros.sum(1)
total_macros = total_macros.fillna(0)
total_macros = total_macros.rename(columns={1:'RECUPERADOS',2:'OBITOS',3:'ATIVOS'})
total_macros = total_macros[['CASOS','OBITOS','RECUPERADOS','ATIVOS']]

total_macros.index = [ macros_idx_labels[x] if isinstance(x,int) else x for x in total_macros.index]

total_macros.astype(float)

evolucao,CASOS,OBITOS,RECUPERADOS,ATIVOS
Leste,503.843,13.353,336.748,153.742
Oeste,222.799,4.382,190.691,27.726
Noroeste,175.313,3.730,129.774,41.809
Norte,181.934,4.757,96.750,80.427
Fora,5.888,148,2.215,3.525


In [25]:
total_regionais = casosct.groupby(['rs','regional','evolucao'])[['sexo']].count().unstack()
total_regionais.columns = total_regionais.columns.droplevel(0)

total_regionais['CASOS'] = total_regionais.sum(1)
total_regionais = total_regionais.fillna(0)
total_regionais = total_regionais.rename(columns={1:'RECUPERADOS',2:'OBITOS',3:'ATIVOS'})
total_regionais = total_regionais[['CASOS','OBITOS','RECUPERADOS','ATIVOS']]

total_regionais.astype(float)

,evolucao,CASOS,OBITOS,RECUPERADOS,ATIVOS
rs,regional,,,,
1,Paranaguá,34.069,845,18.042,15.182
2,Metropolitana,322.992,9.358,226.512,87.122
3,Ponta Grossa,63.432,1.519,42.008,19.905
4,Irati,15.186,262,10.953,3.971
5,Guarapuava,34.080,692,19.799,13.589
6,União da Vitória,11.170,214,8.203,2.753
7,Pato Branco,28.998,651,25.083,3.264
8,Francisco Beltrão,39.171,661,33.250,5.260
9,Foz do Iguaçu,55.107,1.152,42.634,11.321


In [26]:
total_municipios = casosct.groupby(['macro','rs','regional','ibge7','municipio_pr','evolucao'])[['sexo']].count().unstack()

total_municipios[('','CASOS')] = total_municipios.sum(1)
total_municipios = total_municipios.fillna(0).astype(float)
total_municipios.columns = total_municipios.columns.droplevel(0)
total_municipios = total_municipios.rename(columns={1:'RECUPERADOS',2:'OBITOS',3:'ATIVOS'})
total_municipios = total_municipios[['CASOS','OBITOS','RECUPERADOS','ATIVOS']]

total_municipios.index = total_municipios.index.set_levels([ macros_idx_labels[x] if isinstance(x,int) else x for x in total_municipios.index.levels[0]], level=0)

total_municipios

evolucao                                         CASOS  OBITOS  RECUPERADOS  \
macro rs regional  ibge7   municipio_pr                                       
Leste 1  Paranaguá 4101200 Antonina              2.281      58        2.043   
                   4109500 Guaraqueçaba            598      15          482   
                   4109609 Guaratuba             4.266     150        3.863   
                   4115705 Matinhos              2.739      97        2.345   
                   4116208 Morretes              1.931      49        1.579   
...                                                ...     ...          ...   
Norte 22 Ivaiporã  4122172 Rio Branco do Ivaí      259       8          231   
                   4122651 Rosário do Ivaí         379       2          359   
                   4123857 Santa Maria do Oeste    627      22          494   
                   4125001 São João do Ivaí        840      16          744   
Fora  99 Fora      9999999 Fora                  5.888     148        2.215   

evolucao                                         ATIVOS  
macro rs regional  ibge7   municipio_pr                  
Leste 1  Paranaguá 4101200 Antonina                 180  
                   4109500 Guaraqueçaba             101  
                   4109609 Guaratuba                253  
                   4115705 Matinhos                 297  
                   4116208 Morretes                 303  
...                                                 ...  
Norte 22 Ivaiporã  4122172 Rio Branco do Ivaí        20  
                   4122651 Rosário do Ivaí           18  
                   4123857 Santa Maria do Oeste     111  
                   4125001 São João do Ivaí          80  
Fora  99 Fora      9999999 Fora                   3.525  

[400 rows x 4 columns]

In [27]:
ministerio_cols = ['rs','municipio_pr','CASOS','OBITOS']
total_municipios.reset_index()[ministerio_cols].sort_values(['rs','municipio_pr']).to_excel(join(default_output,'ministerio.xlsx'),index=False)

In [28]:
casosct['regional'] = casosct['rs'].apply(lambda x: str(x).zfill(2)) +'ª '+ casosct['regional']
casos_confirmados = casosct[['id_notifica','macro','regional','ibge_resid','municipio','ibge_atend','nome','sexo','idade', 'laboratorio', 'dt_diag', 'comunicacao', 'is', 'evolucao','data_evolucao', 'data_com_evolucao']].copy()
casos_confirmados.loc[casos_confirmados['evolucao']==1,'evolucao'] = 'CURA'
casos_confirmados.loc[casos_confirmados['evolucao']==2,'evolucao'] = 'OBITO'
casos_confirmados.loc[casos_confirmados['evolucao']==3,'evolucao'] = 'ATIVO'

In [29]:
obitos_confirmados = casos_confirmados.loc[casos_confirmados['evolucao']=='OBITO']

In [ ]:
writer = pd.ExcelWriter(join(default_output, "casos_confirmados.xlsx"),
                    engine='xlsxwriter',
                    datetime_format='dd/mm/yyyy',
                    date_format='dd/mm/yyyy')

total.to_excel(writer,'total')
total_macros.to_excel(writer,'total_macros')
total_regionais.to_excel(writer,'total_regionais')
total_municipios.to_excel(writer,'total_municipios')

fit_cols(writer,total,'total')
fit_cols(writer,total_macros,'total_macros')
fit_cols(writer,total_regionais,'total_regionais')
fit_cols(writer,total_municipios,'total_municipios')

for idx, macro in macros_idx_labels.items():
    casos_macro = casos_confirmados.loc[casos_confirmados['macro']==idx].copy()
    casos_macro['macro'] = macro
    casos_macro.to_excel(writer,macro)
    fit_cols(writer,casos_macro,macro)


writer.save()

In [ ]:
writer = pd.ExcelWriter(join(default_output, "obitos_confirmados.xlsx"),
                    engine='xlsxwriter',
                    datetime_format='dd/mm/yyyy',
                    date_format='dd/mm/yyyy')

total.to_excel(writer,'total')
total_macros.to_excel(writer,'total_macros')
total_regionais.to_excel(writer,'total_regionais')
total_municipios.to_excel(writer,'total_municipios')

fit_cols(writer,total,'total')
fit_cols(writer,total_macros,'total_macros')
fit_cols(writer,total_regionais,'total_regionais')
fit_cols(writer,total_municipios,'total_municipios')

for idx, macro in macros_idx_labels.items():
    casos_macro = obitos_confirmados.loc[obitos_confirmados['macro']==idx].copy()
    casos_macro['macro'] = macro
    casos_macro.to_excel(writer,macro)
    fit_cols(writer,casos_macro,macro)


writer.save()

In [ ]:
writer = pd.ExcelWriter(join(default_output,"casos_confirmados_regionais.xlsx"),
                    engine='xlsxwriter',
                    datetime_format='dd/mm/yyyy',
                    date_format='dd/mm/yyyy')

total.to_excel(writer,'total')
total_macros.to_excel(writer,'total_macros')
total_regionais.to_excel(writer,'total_regionais')
total_municipios.to_excel(writer,'total_municipios')

fit_cols(writer,total,'total')
fit_cols(writer,total_macros,'total_macros')
fit_cols(writer,total_regionais,'total_regionais')
fit_cols(writer,total_municipios,'total_municipios')

for regional in sorted(casos_confirmados['regional'].unique()):
    casos_regional = casos_confirmados.loc[casos_confirmados['regional']==regional].copy()
    casos_regional['macro'] = casos_regional.apply(lambda x: macros_idx_labels[x])
    casos_regional.to_excel(writer,regional)
    fit_cols(writer,casos_regional,regional)


writer.save()

In [ ]:
writer = pd.ExcelWriter(join(default_output,"obitos_confirmados_regionais.xlsx"),
                    engine='xlsxwriter',
                    datetime_format='dd/mm/yyyy',
                    date_format='dd/mm/yyyy')

total.to_excel(writer,'total')
total_macros.to_excel(writer,'total_macros')
total_regionais.to_excel(writer,'total_regionais')
total_municipios.to_excel(writer,'total_municipios')

fit_cols(writer,total,'total')
fit_cols(writer,total_macros,'total_macros')
fit_cols(writer,total_regionais,'total_regionais')
fit_cols(writer,total_municipios,'total_municipios')

for regional in sorted(obitos_confirmados['regional'].unique()):
    obitos_regional = obitos_confirmados.loc[obitos_confirmados['regional']==regional].copy()
    obitos_regional['macro'] = obitos_regional.apply(lambda x: macros_idx_labels[x])
    obitos_regional.to_excel(writer,regional)
    fit_cols(writer,obitos_regional,regional)


writer.save()